In [4]:
# import requests
import grpc
import json
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
gRPR_url = '3eec1f.hubs.neynar.com:2283'
api_key = '85C795F9-69DF-4BC1-ABFF-4340B0F9C57E'
HTTP_url = 'https://hub-api.neynar.com'


In [ ]:
def fetch_all_data(base_url, params, data_key, max_requests_per_minute=185):
    """Fetches all pages of data from a paginated API.

    Args:
        base_url: The base URL of the API endpoint.
        params: A dictionary of initial query parameters.
        data_key: The key in the response JSON that contains the data array.

    Returns:
        A list containing all the data objects from all pages.
    """
    all_data = []
    next_page_token = None
    request_count = 0
    start_time = time.monotonic()

    while True:
        if next_page_token:
            params["pageToken"] = next_page_token

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad responses

        data = response.json()
        all_data.extend(data.get(data_key, []))

        next_page_token = data.get("nextPageToken")

        request_count += 1
        elapsed_time = time.monotonic() - start_time
        if request_count >= max_requests_per_minute:
            time_to_sleep = 60 - elapsed_time  # Seconds remaining in the minute
            if time_to_sleep > 0:
                time.sleep(time_to_sleep)
            request_count = 0
            start_time = time.monotonic()

        if not next_page_token:
            break

    return all_data